<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-16 14:47:00
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: -76.72 K (-0.54%)
Current PnL: -26.73 L (-17.27%)
CY Booked + Current PnL: -12.44 L (-8.04%)
-------------------
Total profit:  1.37 L
Total loss:  -28.11 L
-------------------
Total Booked + Current PnL: 14.54 L (11.37%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.18%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.37 L (65.12%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.19%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.88,-18.02,22.09,0.09,18251.0,-18155.0,82622.0,91.68,35.0,M-SC,2.34,253.0,-0.99,0.59,8.55,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.54,9.24,9.57,19.69,20045.0,17718.0,209460.0,-1.35,59.0,X-MC,2.54,78.0,0.88,1.50,16.67,XY25,NTT,AC
39,INDIGOPNTS,1408.00,-1.88,-13.34,15.41,0.02,23312.0,-23279.0,151280.0,130.47,49.0,M-SC,14.79,234.0,-1.00,1.08,31.59,OX40N,NTT,PAINTS
50,MASFIN,398.61,-2.48,-2.51,25.19,22.05,24061.0,-2460.0,95520.0,-15.99,53.0,H-SC,7.23,164.0,-0.10,0.68,38.85,XR,ATH,FINANCE
43,ITC,452.00,-0.10,-1.50,12.47,10.78,29519.0,-3599.0,236719.0,-41.15,46.0,X-LC,1.95,5.0,-0.12,1.70,3.66,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,CERA,9475.00,2.99,-30.25,77.62,23.89,95235.0,-53209.0,122694.0,-32.70,39.0,H-SC,10.58,157.0,-0.56,0.88,5.96,OX40N,NTT,CERAMICS
25,FINCABLES,1641.55,2.83,-1.25,110.54,107.91,161172.0,-1840.0,145804.0,-19.74,60.0,M-SC,12.52,216.0,-0.01,1.04,8.71,OX40N,ATH,CABLES
6,ASIANTILES,137.00,1.85,-16.97,118.81,81.67,92538.0,-15923.0,77887.0,7000.00,45.0,L-SC,12.36,271.0,-0.17,0.56,50.83,XR,NTT,CERAMICS
44,JCHAC,2282.00,1.69,-33.84,51.14,-0.01,43241.0,-43251.0,84554.0,17204.35,39.0,M-SC,7.32,233.0,-1.00,0.61,8.22,OX40N,NTT,AC
61,ROUTE,2227.21,1.41,-47.21,220.21,69.05,154700.0,-62815.0,70251.0,-57.43,52.0,H-SC,22.89,140.0,-0.41,0.50,8.34,SR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJOOENG,143.10,-2.80,-35.68,117.05,39.61,77171.0,-36570.0,65930.0,-56.41,35.0,H-SC,28.73,135.0,-0.47,0.47,3.88,AR,ATH,MISC
18,COFFEEDAY,80.00,-2.55,-39.87,124.97,35.27,85323.0,-45274.0,68275.0,-53.72,49.0,L-SC,10.70,270.0,-0.53,0.49,67.11,XR,NTT,HOTELS
50,MASFIN,398.61,-2.48,-2.51,25.19,22.05,24061.0,-2460.0,95520.0,-15.99,53.0,H-SC,7.23,164.0,-0.10,0.68,38.85,XR,ATH,FINANCE
21,DEN,75.00,-2.38,-40.84,140.15,42.07,70030.0,-34496.0,49968.0,78.76,46.0,M-SC,7.81,235.0,-0.49,0.36,6.12,AR,NTT,ENTERTAINMENT
63,SFL,1287.00,-2.21,-44.24,127.08,26.63,185819.0,-115996.0,146222.0,10.73,25.0,M-SC,11.67,239.0,-0.62,1.05,0.00,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,2.83,-1.25,110.54,107.91,161172.0,-1840.0,145804.0,-19.74,60.0,M-SC,12.52,216.0,-0.01,1.04,8.71,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.88,-13.34,15.41,0.02,23312.0,-23279.0,151280.0,130.47,49.0,M-SC,14.79,234.0,-1.00,1.08,31.59,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.88,-18.02,22.09,0.09,18251.0,-18155.0,82622.0,91.68,35.0,M-SC,2.34,253.0,-0.99,0.59,8.55,OX40N,NTT,DURABLES
66,SIS,528.00,0.20,-24.07,59.71,21.26,50337.0,-26729.0,84303.0,1989.12,50.0,H-SC,5.11,166.0,-0.53,0.60,13.90,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.71,-24.98,51.27,13.47,103716.0,-67374.0,202293.0,-69.09,35.0,H-SC,5.20,158.0,-0.65,1.45,2.86,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-1.08,6.19,62.22,72.27,111960.0,10494.0,179942.0,-8.8,62.0,M-LC,2.45,99.0,0.09,1.29,12.75,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.08,6.19,62.22,72.27,111960.0,10494.0,179942.0,-8.80,62.0,M-LC,2.45,99.0,0.09,1.29,12.75,XR,NTT,IT
25,FINCABLES,1641.55,2.83,-1.25,110.54,107.91,161172.0,-1840.0,145804.0,-19.74,60.0,M-SC,12.52,216.0,-0.01,1.04,8.71,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.45,-4.01,115.37,106.72,136582.0,-4950.0,118386.0,-52.54,35.0,H-SC,2.99,139.0,-0.04,0.85,18.43,AR,ATH,MISC
1,ABB,7934.00,-0.89,-3.90,51.47,45.57,129408.0,-10195.0,251424.0,-38.27,56.0,H-MC,4.56,120.0,-0.08,1.80,6.98,AR,NTT,ELECTRICAL
37,IEX,219.00,-1.36,-4.73,56.01,48.63,106697.0,-9458.0,190496.0,-36.71,46.0,H-SC,14.07,136.0,-0.09,1.36,6.28,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.33,-18.22,90.11,55.47,179282.0,-44327.0,198959.0,-27.60,25.0,X-MC,12.93,64.0,-0.25,1.42,1.94,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-1.16,-9.85,42.10,28.11,76443.0,-19835.0,181575.0,-34.62,27.0,X-MC,12.94,55.0,-0.26,1.30,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.53,-29.23,73.76,22.98,117923.0,-66024.0,159874.0,-26.74,29.0,X-LC,6.80,3.0,-0.56,1.14,0.55,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-1.06,-38.77,120.83,35.20,95189.0,-49891.0,78779.0,3.25,31.0,X-SC,16.12,92.0,-0.52,0.56,0.07,X40,NTT,FOOTWEAR
8,AWL,485.00,-1.16,-22.94,99.18,53.50,230635.0,-69209.0,232542.0,-62.74,31.0,X-MC,2.80,58.0,-0.30,1.67,3.29,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.57,-1.61,26.51,24.48,65691.0,-4059.0,247797.0,-81.32,43.0,X-SC,0.31,86.0,-0.06,1.77,21.15,X40N,NTT,MISC
43,ITC,452.0,-0.10,-1.50,12.47,10.78,29519.0,-3599.0,236719.0,-41.15,46.0,X-LC,1.95,5.0,-0.12,1.70,3.66,X40,NTT,FMCG
33,HINDUNILVR,2922.0,-0.38,-11.12,27.89,13.67,63723.0,-28583.0,228480.0,-17.78,32.0,X-LC,2.14,14.0,-0.45,1.64,7.86,XY25,NTT,FMCG
36,ICICIPRULI,790.0,-1.39,5.82,23.53,30.73,49055.0,11472.0,208477.0,-18.97,58.0,X-MC,2.25,75.0,0.23,1.49,19.33,X40,ATH,INSURANCE
78,UNITDSPR,1644.0,0.37,6.35,13.56,20.77,33765.0,14860.0,249004.0,-2.93,57.0,X-MC,2.44,62.0,0.44,1.78,13.70,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.08,-1.16,-9.85,42.10,28.11,76443.0,-19835.0,181575.0,-34.62,27.0,X-MC,12.94,55.0,-0.26,1.30,0.00,X40,ATH,APPARELS
3,ACC,3906.00,-0.32,-25.53,120.43,64.15,213402.0,-60751.0,177200.0,-56.31,34.0,X-SC,2.60,82.0,-0.28,1.27,0.02,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-1.06,-38.77,120.83,35.20,95189.0,-49891.0,78779.0,3.25,31.0,X-SC,16.12,92.0,-0.52,0.56,0.07,X40,NTT,FOOTWEAR
75,TMPV,600.00,-0.53,-29.23,73.76,22.98,117923.0,-66024.0,159874.0,-26.74,29.0,X-LC,6.80,3.0,-0.56,1.14,0.55,XY24,NTT,AUTO
59,RELAXO,1176.00,-0.41,-47.86,192.10,52.31,145399.0,-69471.0,75689.0,-44.44,41.0,X-SC,6.93,91.0,-0.48,0.54,1.31,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.37,-29.25,101.90,42.84,46866.0,-19014.0,45992.0,-53.72,44.0,X-SC,35.52,83.0,-0.41,0.33,4.05,XY24,NTT,MISC
59,RELAXO,1176.00,-0.41,-47.86,192.10,52.31,145399.0,-69471.0,75689.0,-44.44,41.0,X-SC,6.93,91.0,-0.48,0.54,1.31,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.06,-38.77,120.83,35.20,95189.0,-49891.0,78779.0,3.25,31.0,X-SC,16.12,92.0,-0.52,0.56,0.07,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.45,-4.56,30.01,24.08,36360.0,-5792.0,121158.0,-12.23,33.0,X-SC,5.11,89.0,-0.16,0.87,14.16,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.21,-16.51,71.66,43.32,97443.0,-26892.0,135980.0,-28.31,37.0,X-SC,4.74,90.0,-0.28,0.97,4.41,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.78,-11.88,36.98,20.71,123258.0,-44927.0,333310.0,-23.95,59.0,X-LC,4.46,1.0,-0.36,2.39,11.37,X40,ATH,IT
41,INFY,1972.00,-1.01,8.27,23.99,34.24,81272.0,25888.0,338776.0,-15.65,58.0,X-LC,4.00,2.0,0.32,2.43,17.24,X40,NTT,IT
75,TMPV,600.00,-0.53,-29.23,73.76,22.98,117923.0,-66024.0,159874.0,-26.74,29.0,X-LC,6.80,3.0,-0.56,1.14,0.55,XY24,NTT,AUTO
81,VBL,671.64,-0.13,-3.65,40.81,35.67,124195.0,-11516.0,304326.0,-15.14,56.0,X-LC,4.13,4.0,-0.09,2.18,9.70,X40N,ATH,FMCG
43,ITC,452.00,-0.10,-1.50,12.47,10.78,29519.0,-3599.0,236719.0,-41.15,46.0,X-LC,1.95,5.0,-0.12,1.70,3.66,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.85,-37.69,113.27,32.89,54492.0,-29098.0,48108.0,-694.37,53.0,L-MC,5.96,259.0,-0.53,0.34,32.49,XR,NTT,BANKS
6,ASIANTILES,137.00,1.85,-16.97,118.81,81.67,92538.0,-15923.0,77887.0,7000.00,45.0,L-SC,12.36,271.0,-0.17,0.56,50.83,XR,NTT,CERAMICS
50,MASFIN,398.61,-2.48,-2.51,25.19,22.05,24061.0,-2460.0,95520.0,-15.99,53.0,H-SC,7.23,164.0,-0.10,0.68,38.85,XR,ATH,FINANCE
13,BSOFT,831.70,-1.58,-22.42,94.37,50.79,102971.0,-31539.0,109114.0,-0.66,64.0,H-SC,6.34,151.0,-0.31,0.78,27.04,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.88,-13.34,15.41,0.02,23312.0,-23279.0,151280.0,130.47,49.0,M-SC,14.79,234.0,-1.00,1.08,31.59,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.7,-1.58,-22.42,94.37,50.79,102971.0,-31539.0,109114.0,-0.66,64.0,H-SC,6.34,151.0,-0.31,0.78,27.04,XR,ATH,IT
84,WIPRO,420.0,-1.08,6.19,62.22,72.27,111960.0,10494.0,179942.0,-8.80,62.0,M-LC,2.45,99.0,0.09,1.29,12.75,XR,NTT,IT
41,INFY,1972.0,-1.01,8.27,23.99,34.24,81272.0,25888.0,338776.0,-15.65,58.0,X-LC,4.00,2.0,0.32,2.43,17.24,X40,NTT,IT
27,GLAXO,3466.2,0.72,4.24,33.49,39.15,69568.0,8444.0,207728.0,-16.38,55.0,X-MC,10.68,60.0,0.12,1.49,34.47,X40,ATH,PHARMA
1,ABB,7934.0,-0.89,-3.90,51.47,45.57,129408.0,-10195.0,251424.0,-38.27,56.0,H-MC,4.56,120.0,-0.08,1.80,6.98,AR,NTT,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.23
1,25,44.83
2,50,76.63


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.93
MC    30.99
LC    24.07
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.69
X40      22.98
X40N     13.06
XR        9.75
XY25      9.10
AR        9.06
OX40N     7.59
SR        0.99
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.52
H-SC    23.77
X-LC    20.64
M-SC    11.70
X-SC     8.04
H-MC     4.81
L-SC     1.42
M-MC     1.32
M-LC     1.29
H-LC     1.13
L-LC     1.01
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.41,-7.37,41.90
IT,13.00,-15.80,74.03
FINANCE,9.46,-20.59,73.04
MISC,7.25,-28.19,80.62
ELECTRICAL,6.02,-11.54,52.40
PAINTS,5.85,-13.04,29.83
INSURANCE,4.75,-2.21,37.18
PHARMA,4.14,-1.83,33.88
AUTO,2.80,-33.70,80.05


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3195770.0,21
AR,1307796.0,10
XR,1298496.0,13
X40,1014681.0,14
X40N,859762.0,9
OX40N,731050.0,10
XY25,370900.0,6
SR,281800.0,2
MH,76704.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3658108.0,25
M-SC,1424375.0,15
X-MC,1403079.0,16
X-LC,845197.0,11
X-SC,794454.0,8
H-MC,406647.0,3
L-SC,263102.0,3
M-LC,111960.0,1
H-LC,76125.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1282099.0      6
           AR         894049.0      5
M-SC       XY24       821531.0      6
H-SC       XR         781423.0      7
X-MC       X40        489258.0      7
           XY24       403477.0      3
H-SC       OX40N      342033.0      4
X-LC       X40        336130.0      5
X-MC       X40N       333785.0      4
X-SC       X40N       308533.0      3
M-SC       OX40N      303776.0      5
X-SC       XY24       296628.0      3
H-SC       SR         281800.0      2
X-LC       X40N       217444.0      2
H-MC       AR         211314.0      2
X-LC       XY24       196702.0      2
H-MC       XY24       195333.0      1
X-SC       X40        189293.0      2
L-SC       XR         177861.0      2
X-MC       XY25       176559.0      2
M-SC       XR         172760.0      2
           AR         126308.0      2
M-LC       XR         111960.0      1
X-LC       XY25        94921.0      2
L-SC       OX40N       85241.0      1
H-SC       MH          76704.0      1
H-LC       AR          76125.0      1
M-MC       XY25        60959.0      1
L-MC       XR          54492.0      1
L-LC       XY25        38461.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
